In [ ]:
import pandas as pd
import os
import numpy as np
import glob
import matplotlib.pyplot as plt
from scipy.stats import spearmanr

In [ ]:
# Importing the data frame
main_df = pd.read_excel('Selected_Springs_final_updated.xlsx')
main_df

In [ ]:
#Calculated the distance between each spring coordinate and the closest grid point

storage_df = pd.read_csv('GWS.csv')
pd.set_option('display.max_columns', None)

# Defined the Haversine distance function
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth's radius in kilometers
    lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

# Found the closest storage latitude-longitude pairs for each spring
def closest_storage(spring_row):
    spring_lat = spring_row['Latitude']
    spring_lon = spring_row['Longitude']
    distances = storage_df.apply(lambda row: haversine(spring_lat, spring_lon, row['lat'], row['lon']), axis=1)
    closest_storage_index = distances.idxmin()
    closest_row = storage_df.iloc[closest_storage_index]
    distance = distances[closest_storage_index]
    return pd.Series([closest_row['lon'], closest_row['lat'], distance])

# Applied the function to the spring data frame and create the output data frame
output_columns = ['longitude', 'latitude', 'distance'] 
output_df = main_df.apply(closest_storage, axis=1, result_type='expand')
output_df.columns = output_columns


# Saved the output data frame
#output_df.to_csv('output.csv', index=False)

In [ ]:
output_df['Start date'] = main_df['Start date']
output_df['End date'] = main_df['End date']
output_df['Name'] = main_df['Name']
output_df['WoKAS ID'] = main_df['WoKaS_ID']
output_df.to_csv('Spring New Coordinates.csv', index=None)
output_df

In [ ]:
storage_df = pd.read_csv('GWS.csv')


In [ ]:
import pandas as pd


# Step 1: Extracted latitude and longitude values from output_df
latitude_list = output_df['latitude'].tolist()
longitude_list = output_df['longitude'].tolist()

# Step 2: Converted "Start date" and "End date" columns to datetime objects
output_df['Start date'] = pd.to_datetime(output_df['Start date'])
output_df['End date'] = pd.to_datetime(output_df['End date'])

# Step 3: Converted the time series columns of storage_df to datetime objects
time_columns = storage_df.columns[2:]  # Excluding 'lat' and 'lon' columns

# Step 4: Filtered storage_df based on latitude, longitude, and time period
subset_list = []

for lat, lon, start_date, end_date in zip(latitude_list, longitude_list, output_df['Start date'], output_df['End date']):
    # Updated the start_date to the first day of the month if not already
    if start_date.day != 1:
        start_date = start_date.replace(day=1)

    # If end_date is the first day of the month, update it to the second day
    if end_date.day == 1:
        end_date = end_date.replace(day=2)
        
    start_date_str = start_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')
    
    filtered_df = storage_df.loc[(storage_df['lat'] == lat) & (storage_df['lon'] == lon), time_columns]
    
    # Filtered columns based on start_date_str and end_date_str
    subset = filtered_df.loc[:, start_date_str:end_date_str]
    subset_list.append(subset)

# Step 5: Retrieved the subset of storage_df with desired columns
result_df = pd.concat(subset_list, axis=0)
result_df.reset_index(drop=True, inplace=True)
result_df

# result_df should now contain the desired subset of storage_df

In [ ]:
folder_path = './'

In [ ]:
#THIS PARTICULAR CODE CALCULATES THE MONTHLY MEAN OF DISCHARGE AND USES LINEAR INTERPOLATION 
#FOR THE VALUES IN THE DATAFRAMES WITH SOME MISSING MONTHS
#I THINK THIS IS MORE FAVORABLE

import os
import pandas as pd


for index, row in main_df.iterrows():
    input_file_name = row['WoKaS_ID'] + '.csv'
    input_file_path = os.path.join(folder_path, input_file_name)

    output_file_name = 'converted_' + input_file_name

    df = pd.read_csv(input_file_path, skiprows=8, encoding='latin1')
    df[['Timestamp', 'discharge']] = df['Timestamp\tdischarge'].str.split('\t', expand=True)
    df.drop(columns=['Timestamp\tdischarge'], inplace=True)

    df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%d.%m.%Y %H:%M:%S')
    df.set_index('Timestamp', inplace=True)

    start_date = row['Start date']
    end_date = row['End date']

    filtered_df = df.loc[start_date:end_date]

    filtered_df['discharge'] = pd.to_numeric(filtered_df['discharge'], errors='coerce')
    monthly_discharge_df = filtered_df.resample('M').mean()

    # Performed linear interpolation for missing discharge values
    monthly_discharge_df['discharge'].interpolate(method='linear', inplace=True)

    monthly_discharge_df.reset_index(inplace=True)

    # Retrieved the corresponding row from result_df
    storage_row = result_df.iloc[index]

    # Droped columns containing NaN values in storage_row
    storage_row = storage_row.dropna()

    # Added 'storage' column to monthly_discharge_df and fill with values from storage_row
    monthly_discharge_df = monthly_discharge_df.assign(storage=storage_row.values)

    output_file_path = os.path.join(folder_path, output_file_name)
    monthly_discharge_df.to_csv(output_file_path, index=False)

In [ ]:
folder_path = "./"
files = glob.glob(os.path.join(folder_path, 'converted_*.csv'))

results = []

for file_path in files:
    df = pd.read_csv(file_path)
    spearman_corr, p_value = spearmanr(df['storage'], df['discharge'])
    
    results.append({
        'file_name': os.path.basename(file_path),
        'spearman_corr': spearman_corr,
        'p_value': p_value
    })

Spearman_df = pd.DataFrame(results)
Spearman_df

In [ ]:
#▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶Lagging the Discharge by one Month◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀

In [ ]:
import pandas as pd
import glob

# Specifed the folder path and the file pattern
folder_path = './'
file_pattern = 'converted_*.csv'  # Adjust the file extension if needed

# Got the list of all matching files in the folder
files = glob.glob(f"{folder_path}/{file_pattern}")

# Looped through each file and apply the processing steps
for file in files:
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file)

    # Applied the processing steps as before
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df.set_index('Timestamp', inplace=True)
    df['lagged_discharge'] = df['discharge'].shift(1)#SHIFTED BY ONE MONTH
    avg_lagged_discharge = df['lagged_discharge'].mean()
    df['lagged_discharge'].fillna(avg_lagged_discharge, inplace=True) #THIS REPLACES THE INITIAL DISPLACED VAULE WITH THE MEAN
    df.reset_index(inplace=True)
    df['Timestamp'] = df['Timestamp'].dt.strftime('%Y-%m-%d')

  # Saved the processed DataFrame to a new CSV file with the prefix "lagged" added to the current name
    output_file_name = 'lagged_' + file.split('\\')[-1]
    output_file = f"{folder_path}/{output_file_name}"
    df.to_csv(output_file, index=False)

In [ ]:
import os
import glob
import pandas as pd
from scipy.stats import spearmanr

folder_path = "./"
files = glob.glob(os.path.join(folder_path, 'lagged_*.csv'))

results = []

for file_path in files:
    df = pd.read_csv(file_path)
    spearman_corr, p_value = spearmanr(df['storage'], df['lagged_discharge'])
    
    results.append({
        'file_name': os.path.basename(file_path),
        'spearman_corr': spearman_corr,
        'p_value': p_value
    })

Spearman_df_lagged = pd.DataFrame(results)
Spearman_df_lagged

In [ ]:
#▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶Plotting 1◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀

In [ ]:
Spearman_df['latitude'] = output_df['latitude']
Spearman_df['longitude'] = output_df['longitude']
Spearman_df

In [ ]:
#▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶Plotting◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀

In [ ]:
df = pd.read_csv('converted_FR-0050.csv')

In [ ]:
df.Timestamp = pd.to_datetime(df.Timestamp)
fig, ax1 = plt.subplots(figsize=(15, 7))  # Adjusted the width and height of the graph

# Plotted discharge in red on the left y-axis
ax1.plot(df['Timestamp'], df['discharge'], color='red', label='Discharge')
ax1.tick_params(axis='x', labelsize=15)
ax1.set_xlabel('Timestamp', fontsize=20)
ax1.set_ylabel('Discharge', color='red', fontsize=20)
ax1.tick_params(axis='y', labelcolor='red', labelsize=20)

# Created a second y-axis for storage
ax2 = ax1.twinx()

# Plotted storage in blue on the right y-axis
ax2.plot(df['Timestamp'], df['storage'], color='blue', label='Storage')
ax2.set_ylabel('Storage', color='blue',fontsize=20)
ax2.tick_params(axis='y', labelcolor='blue', labelsize=20 )

# Set the title and show the plot
plt.title('Spring Discharge (FR-0057) and GWS (without soil moisture)', fontsize = 22)
plt.tight_layout()
plt.show()

In [ ]:
df2 = pd.read_csv('lagged_converted_FR-0050.csv')

In [ ]:
df.Timestamp = pd.to_datetime(df.Timestamp)
fig, ax1 = plt.subplots(figsize=(15, 7))  # Adjusted the width and height of the graph

# Plotted discharge in red on the left y-axis
ax1.plot(df['Timestamp'], df2['lagged_discharge'], color='red', label='Discharge')
ax1.tick_params(axis='x', labelsize=20)
ax1.set_xlabel('Timestamp', fontsize=20)
ax1.set_ylabel('Discharge', color='red', fontsize=20)
ax1.tick_params(axis='y', labelcolor='red', labelsize=18)

# Created a second y-axis for storage
ax2 = ax1.twinx()

# Plotted storage in blue on the right y-axis
ax2.plot(df['Timestamp'], df2['storage'], color='blue', label='Storage')
ax2.set_ylabel('Storage', color='blue',fontsize=20)
ax2.tick_params(axis='y', labelcolor='blue', labelsize=18 )

# Set the title and show the plot
plt.title('Lagged Spring Discharge (FR-0057) and GWS (without soil moisture)', fontsize = 22)
plt.tight_layout()
plt.show()

In [ ]:
#▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶CLUSTER ANALYSIS OF THE SPRING◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀◀

In [ ]:
#Imported a dataframe containing the springs, their catchment sizes and correlation coefficient

df = pd.read_csv('cluster.csv')
df

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# Assigned a number for each spring class from C1 to C6, this will help in visualization
class_mapping = {'C1': 1, 'C2': 2, 'C3': 3, 'C4': 4, 'C5': 5, 'C6': 6}
df['class_number'] = df['spring_class'].map(class_mapping)
# Created a cluster plot using seaborn library
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='class_number', y='correlation_coefficient', hue='spring_class', palette='deep', s=200) # Increase the value of s to increase point size

plt.xlabel('Spring Class', fontsize=20)
plt.ylabel('Correlation Coefficient', fontsize=20)
plt.title('Cluster Plot of Correlation of Classes of Spring', fontsize=20)

# Customized the x-axis with the class labels
plt.xticks(range(1, 7), ['C1', 'C2', 'C3', 'C4', 'C5', 'C6'])

# Added catchment_size labels for each point
for idx, row in df.iterrows():
    plt.text(row['class_number'] + 0.05, row['correlation_coefficient'], row['catchment_size'], fontsize=10)

plt.show()

In [ ]:
df = pd.read_csv('cluster2.csv')
df

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


# Created a scatter plot using Matplotlib
plt.figure(figsize=(12, 8))
plt.scatter(df['correlation_coefficient'], df['catchment_size'])

# Calculated the trendline
z = np.polyfit(df['correlation_coefficient'], df['catchment_size'], 1)
p = np.poly1d(z)

# Plotted the trendline in red
plt.plot(df['correlation_coefficient'], p(df['correlation_coefficient']), 'r-')

plt.xlabel('Correlation Coefficient', fontsize= 20)
plt.ylabel('Catchment Size (km²)', fontsize= 20)
plt.title('Scatter Plot of Catchment Size vs Correlation Coefficient with Trendline', fontsize= 20)
plt.xticks(fontsize= 20)
plt.yticks(fontsize= 20)

plt.show()

In [ ]:
# Scatter Plot of Catchment Size vs Correlation Coefficient with Trendline

import matplotlib.pyplot as plt
import numpy as np

# Created a scatter plot using Matplotlib
plt.figure(figsize=(10, 6))
plt.scatter(df['correlation_coefficient'], df['catchment_size'])

# Calculated the trendline
z = np.polyfit(df['correlation_coefficient'], df['catchment_size'], 1)
p = np.poly1d(z)

# Plotted the trendline in red
plt.plot(df['correlation_coefficient'], p(df['correlation_coefficient']), 'r-')

# Added text labels for each point
for i, txt in enumerate(df['WoKAS ID']):
    plt.annotate(txt, (df['correlation_coefficient'][i], df['catchment_size'][i]))

plt.xlabel('Correlation Coefficient')
plt.ylabel('Catchment Size')
plt.title('Scatter Plot of Catchment Size vs Correlation Coefficient with Trendline')

plt.show()